# Imports

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,IntegerType, StructType,StringType
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql import functions as sf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col , split

# Load Dataset

In [105]:
df_manager = pd.read_csv('/home/user/Desktop/files_desktop/DATA_managers/adm2018.csv')

In [106]:
df_manager.head()

,anio,expediente,ingresos,activos,patrimonio,utilidad_perdida,cantidad_epleados,fecha_ejecucion,fecha_actualizacion,adm_cedula,administrador,adm_fnombramiento,adm_ccargo,cargo,adm_periodo,adm_ftermino,adm_frmercantil
0,2018,1,142310.91,1337788.57,1295310.23,10245.38,12.0,2019-06-18 02:32:57.053000,2019-06-18 03:17:01.126000,1701093039,LLERENA ALBUJA TERESA,2008-03-29 15:13:33.886000,1,GERENTE GENERAL,4.0,2012-03-03 16:28:21.526000,2008-04-08 00:00:00
1,2018,10000,39118392.90,27482362.90,7449106.56,1530469.95,1858.0,2019-06-18 02:34:47.153000,2019-06-18 03:17:02.703000,1706450655,MONTALVO PAREDES PABLO SANTIAGO,2014-07-29 08:50:29.166000,32,PRESIDENTE DEL DIRECTORIO,3.0,2016-01-04 10:26:39.920000,2014-08-07 00:00:00
2,2018,100009,1275257.93,488437.09,147892.77,585.75,111.0,2019-06-18 02:46:54.700000,2019-06-18 03:17:17.590000,0922066337,DOMINGUEZ ESPINOZA STEPHANIA ELIZABETH,2017-02-15 12:24:44.593000,30,PRESIDENTE,2.0,NaN,2017-02-17 00:00:00
3,2018,100010,207421.36,1143468.72,133697.10,30227.61,5.0,2019-06-18 02:46:54.753000,2019-06-18 03:17:17.590000,0902954627,WONG LAMA OTTON FERNANDO,2000-02-24 00:00:00,1,GERENTE GENERAL,5.0,2008-01-28 16:26:43.083000,2000-03-01 00:00:00
4,2018,100022,559598.53,292423.91,179964.99,-848.55,22.0,2019-06-18 02:46:53.850000,2019-06-18 03:17:17.593000,0924381296,GORDILLO NEIRA JAIME ARTURO,2000-02-14 00:00:00,1,GERENTE GENERAL,5.0,2006-04-28 11:23:25.576000,2000-03-01 00:00:00


In [107]:
df_manager.isna().sum()

anio                        0
expediente                  0
ingresos                    0
activos                     0
patrimonio                  0
utilidad_perdida            0
cantidad_epleados           0
fecha_ejecucion             0
fecha_actualizacion         0
adm_cedula                  0
administrador             780
adm_fnombramiento           0
adm_ccargo                  0
cargo                       0
adm_periodo              2276
adm_ftermino           117444
adm_frmercantil            25
dtype: int64

# Fix columns format

In [108]:
df_manager["adm_ftermino"].fillna("2019-06-18", inplace=True)
df_manager['date_begin'] = pd.to_datetime((df_manager.adm_fnombramiento.str.split().str[0]), infer_datetime_format=True)
df_manager['date_end'] = pd.to_datetime((df_manager.adm_ftermino.str.split().str[0]), infer_datetime_format=True)

# Filter dataset

- Part1: Records between the years 2014 -2018
- Part2: Records of those who started working outside the range of years (2014-2018) and finished their work activities within the range of years analyzed.
- Part3: Records of those who started working between the range of years (2014-2018) and until now they are working (2019).
- Part4: Records of those who started working outside the range of years (2014-2018) and until now they are working (2019).

In [114]:

part1 = df_manager[  ((df_manager.date_end < '2019-01-01') & (df_manager.date_end >= '2014-01-01')) & 
((df_manager.date_begin < '2019-01-01') & (df_manager.date_begin >= '2014-01-01'))]
part2 = df_manager[  ((df_manager.date_end < '2019-01-01') & (df_manager.date_end >= '2014-01-01')) & 
( (df_manager.date_begin < '2014-01-01') )]

part3 = df_manager[  ( (df_manager.date_end == '2019-06-18') ) & 
((df_manager.date_begin < '2019-01-01') & (df_manager.date_begin >= '2014-01-01'))]
part4 = df_manager[  ( (df_manager.date_end == '2019-06-18') ) & 
( (df_manager.date_begin < '2014-01-01') )]

df_final = pd.concat([part1, part2, part3, part4 ], ignore_index=True)

# Load Data of companies

In [2]:
df_firms = dd.read_csv('/home/user/Desktop/files_desktop/DATA_managers/BASE.csv')

In [4]:
# spark = SparkSession.builder \
#     .master('local[*]') \
#     .config("spark.driver.memory", "20g") \
#     .appName('MyFirstCSVLoad') \
#     .getOrCreate()
# super_18 = spark.read.format("csv").load("/home/user/Desktop/files_desktop/DATA_managers/BASE.csv", header = 'true')

22/05/04 14:40:45 WARN Utils: Your hostname, user-Inspiron-5570 resolves to a loopback address: 127.0.1.1; using 192.168.200.10 instead (on interface wlp2s0)
22/05/04 14:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/04 14:40:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
